In [1]:
from typing import Annotated

In [2]:
# Suppose we have a TypedDict that also has one parameter as email.
# We want to make sure that the email is a valid email address.
# Like it shouldn't be empty, and it should contain '@' and '.' etc rules.

email = Annotated[str, "Must be a valid email address"]

In [3]:
print(email.__metadata__)

('Must be a valid email address',)


In [ ]:
from typing import Sequence

# Suppose we have a TypedDict that also has one parameter as a list of emails.
emails = Annotated[Sequence[str], "Must be a list of valid email addresses"]
print(emails.__metadata__)

('Must be a list of valid email addresses',)


In **LangGraph**, a **reducer** is used to **accumulate or merge message history** during state transitions. This is particularly useful in scenarios where you want to manage how messages are stored and processed in your graph.


```python
from langgraph.graph.message import add_messages
```

This is passed like:

```python
messages: Annotated[Sequence[BaseMessage], add_messages]
```

So now you're wondering: **Are there other reducers? What do they do? Can I write my own?**

---

## ✅ Built-in Reducers in LangGraph (as of mid-2024)

LangGraph provides a few **standard reducers** in `langgraph.graph.message`. The main ones include:

### 1. `add_messages`

* 🔧 **Most commonly used reducer**
* 📌 It **appends** new messages to the current list
* Used when you want to **preserve all history**

```python
from langgraph.graph.message import add_messages
```

Usage:

```python
messages: Annotated[Sequence[BaseMessage], add_messages]
```

---

### 2. `replace_messages`

* 🔁 Replaces the old message history with the new one
* Use this when you **only want to keep the latest message(s)**

```python
from langgraph.graph.message import replace_messages
```

Usage:

```python
messages: Annotated[Sequence[BaseMessage], replace_messages]
```

This is useful for **stateless agents** or memory-trimmed versions.

---

### 3. `collect_messages`

* 🧠 Returns a **list of lists**, tracking messages **at each step** (a message trace)
* Useful for debugging, auditing, or visualizing how the conversation evolved

```python
from langgraph.graph.message import collect_messages
```

Usage:

```python
messages: Annotated[list[list[BaseMessage]], collect_messages]
```

---

### 4. (Bonus) Write your own custom reducer 🛠️

You can define your own function with this signature:

```python
def my_reducer(prev_messages, new_messages) -> combined_messages:
    # Your logic here
    return updated_message_list
```

Then pass it in `Annotated`:

```python
messages: Annotated[list[BaseMessage], my_reducer]
```

### Example: Keep only last 2 user+AI messages

```python
def keep_last_two(prev, new):
    combined = prev + new
    return combined[-4:]  # each message round has user+AI
```

---

## 🧠 Summary Table

| Reducer            | What it does                   | Use Case                             |
| ------------------ | ------------------------------ | ------------------------------------ |
| `add_messages`     | Appends all new messages       | Full memory (ReAct style)            |
| `replace_messages` | Overwrites previous            | Stateless or windowed agent          |
| `collect_messages` | Tracks message lists over time | Auditing, visualization              |
| `custom`           | You define the logic           | Anything you want: trim, filter, etc |

